<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
RandomSearch - Keras - Combined Cycle Power Plant
</b></font> </br></p>

---


# 0  | Install & Import
***

In [ ]:
# Install
!uv pip install --system -q keras_tuner

In [ ]:
# Import
from pandas import read_csv, DataFrame, concat

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

import keras
import keras_tuner
from keras_tuner import HyperParameters, RandomSearch
from keras.layers import Dense, Input

from keras.utils import set_random_seed, plot_model

from tensorflow import keras
from tensorflow.config.experimental import enable_op_determinism
import tensorflow as tf

In [ ]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# 1 | Understand
---


<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Aufgabe verstehen</br>
✅ Daten sammeln</br>
✅ Statistische Analyse (Min, Max, Mean, Korrelation, ...)</br>
✅ Datenvisualisierung (Streudiagramm, Box-Plot, ...)</br>
✅ Prepare Schritte festlegen</br>

<p><font color='black' size="5">
Anwendungsfall
</font></p>

---



Der Datensatz enthält 9568 Datenpunkte, die von einem Gas- und Dampfturbinenkraftwerk über einen Zeitraum von 6 Jahren (2006-2011) gesammelt wurden, als das Kraftwerk unter Volllast in Betrieb genommen wurde. Die Funktionen bestehen aus den stündlichen durchschnittlichen Umgebungsvariablen
+ Temperatur (T),
+ Umgebungsdruck (AP),
+ Relative Luftfeuchtigkeit (RH)
+ und Abgasvakuum (V),

um die stündliche Netto-Stromerzeugung (EP) der Anlage vorherzusagen.


Ein GuD-Kraftwerk (GuD-Kraftwerk) setzt sich aus Gasturbinen (GT), Dampfturbinen (ST) und Abhitzedampferzeugern zusammen. Bei einem GuD-Kraftwerk wird der Strom durch Gas- und Dampfturbinen erzeugt, die in einem Kreislauf kombiniert werden, und von einer Turbine auf eine andere übertragen. Während das Vakuum von der Dampfturbine beeinflusst wird und sich auf sie auswirkt, beeinflussen die anderen drei Umgebungsvariablen die GT-Leistung.

Der Hochdruckdampf in einem GuD-Kraftwerk wird im sogenannten Abhitzedampferzeuger (AHE) erzeugt – einem zentralen Bauteil, das die Abwärme der Gasturbine nutzt, um Wasser in Dampf umzuwandeln.

Nach der Expansion des Dampfes in der Dampfturbine muss dieser wieder kondensieren, um erneut im Kreislauf genutzt werden zu können. Das passiert im sogenannten Kondensator, einem Wärmeübertrager, in dem der Dampf durch Kühlwasser (z. B. Fluss- oder Meerwasser) abgekühlt wird.

Beim Kondensieren entsteht ein Unterdruck (Vakuum), weil Wasserdampf beim Übergang in den flüssigen Zustand viel Volumen verliert – und das senkt den Druck im Kondensator erheblich, typischerweise auf etwa 0,05 bar oder sogar weniger (je nach Kühlwassertemperatur).

Das Vakuum erhöht die Druckdifferenz über der Dampfturbine, also:

`Druck am Einlass (Hochdruckdampf)−Druck am Auslass (Vakuum)`


Je größer dieser Unterschied ist, desto mehr Energie kann der Dampf beim Expandieren in der Turbine abgeben.

Dadurch:

+ steigt die mechanische Leistung der Dampfturbine,

+ wird mehr Strom im Generator erzeugt,

+ und der Wirkungsgrad des gesamten GuD-Kraftwerks verbessert sich.

[DataSet](http://archive.ics.uci.edu/dataset/294/combined+cycle+power+plant)    
[Info](http://archive.ics.uci.edu/dataset/294/combined+cycle+power+plant)

In [ ]:
df = read_csv(
    "https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02_daten/05_tabellen/ccpp.csv"
)

In [ ]:
data = df.copy()
target = data.pop("PE")

<p><font color='black' size="5">
EDA (Exploratory Data Analysis) mit Pandas
</font></p>

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
data.corr()

# 2 |  Prepare

---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Nicht benötigte Features löschen</br>
✅ Datentyp ermitteln/ändern</br>
✅ Duplikate ermitteln/löschen</br>
✅ Missing Values behandeln</br>
✅ Ausreißer behandeln</br>
✅ Kategorischer Features Kodieren</br>
✅ Numerischer Features skalieren</br>
✅ Feature-Engineering (neue Features schaffen)</br>
✅ Dimensionalität reduzieren</br>
✅ Resampling (Over-/Undersampling)</br>
✅ Pipeline erstellen/konfigurieren</br>
✅ Train-Test-Split durchführen</br>

<p><font color='black' size="5">
Datentyp ermitteln
</font></p>

In [ ]:
all_col = data.columns
num_col = data.select_dtypes(include="number").columns
cat_col = data.select_dtypes(exclude="number").columns

<p><font color='black' size="5">
Skalierung
</font></p>

In [ ]:
scaler = MinMaxScaler()
data[num_col] = scaler.fit_transform(data[num_col])

<p><font color='black' size="5">
Train-Test-Set
</font></p>


In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.3, random_state=42
)
data_train.shape, data_test.shape, target_train.shape, target_test.shape

# 3 | Modeling
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellauswahl treffen</br>
✅ Pipeline erweitern/konfigurieren</br>
✅ Training durchführen</br>
✅ Hyperparameter Tuning</br>
✅ Cross-Valdiation</br>
✅ Bootstrapping</br>
✅ Regularization</br>

<p><font color='black' size="5">
Zufallszahl initialisieren
</font></p>

In [ ]:
set_random_seed(42)
enable_op_determinism()

<p><font size="5">
Modelauswahl
</p>

In [ ]:
# Hyperparameter-Objekt erstellen
hp = HyperParameters()

In [ ]:
# Funktion, um das Keras-Modell mit variablen Hyperparametern zu erstellen - notwendig für die Verwendung des keras-tuners
def build_model(hp):
    model = keras.Sequential()
    model.add(Input(shape=(4,)))  # Definieren des Input-Layers separat

    # Bestimmen der Anzahl der versteckten Schichten
    num_layers = hp.Int("num_layers", min_value=1, max_value=5, step=1)

    # Schleife, um die jeweilige Anzahl von Schichten hinzuzufügen
    for i in range(num_layers):
        model.add(
            Dense(
                units=hp.Int("units_" + str(i), min_value=50, max_value=1000, step=50),
                activation="relu",
            )
        )

    # Output Layer
    model.add(Dense(1))

    # Kompilieren des Modells
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4, 1e-5])
        ),
        loss="mean_absolute_error",
    )
    return model

<p><font size="5">
Hyperparameter Tuning - RandomSearch
</p>

[keras_tuner](https://keras.io/keras_tuner/)

hp.Int: Hierbei wird ein ganzzahliger Hyperparameter mithilfe des hp-Objekts (Hyperparameter) aus KerasTuner definiert.
+ 'units': Dies ist der Name des Hyperparameters. Er bezieht sich  auf die Anzahl der Einheiten in einer neuronalen Netzwerkschicht, was ein gängiger Hyperparameter in maschinellen Lernmodellen ist.
+ min_value=32: Dieser Wert legt den minimal zulässigen Wert für die Anzahl der Einheiten fest, welcher in diesem Fall 32 beträgt.
+ max_value=512: Hiermit wird der maximal zulässige Wert für die Anzahl der Einheiten definiert, welcher hier 512 ist.
+ step=32: Dies definiert die Schrittweite für die Suche nach möglichen Werten.

Die Anzahl der Einheiten kann nur Werte annehmen, die Vielfache von 32 sind, beginnend bei 32 und endend bei 512 (einschließlich). Die möglichen Werte wären also 32, 64, 96, ..., 512.

hp.Choice: Diese Funktion definiert einen Hyperparameter vom Typ "Auswahl".
+ 'learning_rate': Dies ist der Name des Hyperparameters. Er bezieht sich auf die Lernrate, ein wichtiger Hyperparameter in der Optimierung neuronaler Netze. Die Lernrate steuert die Schrittweite, mit der das Modell während des Trainings seine Parameter anpasst.
+ values=[1e-2, 1e-3, 1e-4]: Diese Liste enthält die möglichen Werte, die der Hyperparameter "learning_rate" annehmen kann.

In diesem Fall sind die möglichen Werte 10^-2, 10^-3 und 10^-4.

In [ ]:
# Keras Tuner Initialisierung
tuner = RandomSearch(
    build_model, objective="val_loss", max_trials=5, project_name="keras_tuner"
)

In [ ]:
tuner.search_space_summary()

In [ ]:
# Tuner-Suche starten
tuner.search(data_train, target_train, epochs=10, validation_split=0.2)

In [ ]:
# Die Top 3 Modelle
best_hps_list = tuner.get_best_hyperparameters(num_trials=3)
for i, hps in enumerate(best_hps_list, start=1):
    num_layers = hps.get("num_layers")  # Anzahl der Layer abrufen
    print(f"Bestes Set {i}:")
    print("Layer:", num_layers)

    # Durch jeden Layer iterieren und die zugehörigen 'units' abrufen
    for j in range(num_layers):
        print(f"  Units in Layer {j + 1}:", hps.get(f"units_{j}"))

    print("Lernrate:", hps.get("learning_rate"))
    print("-" * 30)

In [ ]:
# tuner.results_summary()

# 4 | Evaluate
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Prognose (Train, Test) erstellen</br>
✅ Modellgüte prüfen</br>
✅ Residuenanalyse erstellen</br>
✅ Feature Importance/Selektion prüfen</br>
✅ Robustheitstest erstellen</br>
✅ Modellinterpretation erstellen</br>
✅ Sensitivitätsanalyse erstellen</br>
✅ Kommunikation (Key Takeaways)</br>

<p><font color='black' size="5">
Prognose auf Basis des besten Modells
</font></p>

In [ ]:
# Laden des besten Modells
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
best_model.summary()

In [ ]:
# Verwenden des besten Modells zur Vorhersage
target_pred = best_model.predict(data_test)

<p><font color='black' size="5">
Bestimmtheitsmass
</font></p>

In [ ]:
r2 = r2_score(target_test, target_pred)
print(f"Modell: {best_model} -- Test --- Bestimmtheitsmass: {r2:5.2f}")

<p><font color='black' size="5">
Mean Absolut Error
</font></p>

In [ ]:
mae = mean_absolute_error(target_test, target_pred)
print(f"Modell: {best_model} -- Test -- Mean Absolute Error: {mae:5.2f}")

# 5 | Deploy
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellexport und -speicherung</br>
✅ Abhängigkeiten und Umgebung</br>
✅ Sicherheit und Datenschutz</br>
✅ In die Produktion integrieren</br>
✅ Tests und Validierung</br>
✅ Dokumentation & Wartung</br>